### Model for optimizing player transfers

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pulp import LpVariable, LpProblem, LpMaximize, LpStatus, value, LpMinimize, PULP_CBC_CMD, makeDict, lpSum

In [2]:
# set up parameters
transfer_budget_dict = {'Manchester United':30e6, 'Leeds United':10e6, 'Newcastle United':200e6}
current_year = 2022
current_team = 'Manchester United' 
team_country = 'England'
max_team_size = 25
max_foreign_size = 17
max_in = max_team_size // 4
max_transfer_budget = transfer_budget_dict[current_team]
max_old = 1

In [3]:
# prepare player data
df = pd.read_csv('player_stats_10000.csv', index_col=0)
# add index to name since player names are not unique
df['name'] = df['name'] + ' (' + (np.arange(df.shape[0])).astype(str) + ')' 
# consolidate/simplify positions
df['position'].replace('RW', 'RM', inplace=True)
df['position'].replace('ST', 'CF', inplace=True)
df['position'].replace('LW', 'LM', inplace=True)
df['position'].replace('LWB', 'LB', inplace=True)
df['position'].replace('RWB', 'RB', inplace=True)

# ability function
df['ability'] = np.where(df['age'] < 25,
                         (df['potential'] + df['overall'])/2, 
                         df['overall'])

# flag older players
df['old'] = (df['age'] >= 32).astype(int)

# flag players that have a low probability of transferring
df['low_proba'] = 0
# condition 1: just signed new contract
df.loc[(df['start'] == current_year), 'low_proba'] = 1
# condition 2: on long term contract of 4 or more years 
df.loc[(df['expiration'] - current_year >= 4), 'low_proba'] = 1
# condition 3: "marquee" outfield player
marquee_age_outfield = (df['age'] >= 22) & (df['age'] <= 32) 
marquee_contract = (df['expiration'] - current_year >= 3) 
marquee_ability = (df['ability'] >= np.percentile(df['ability'], 97)) 
df.loc[marquee_age_outfield & marquee_contract & marquee_ability & (df['position'] != 'GK'), 'low_proba'] = 1
# condition 4: "marquee" goalkeeper
marquee_age_gk = (df['age'] >= 25) & (df['age'] <= 35) 
marquee_contract = (df['expiration'] - current_year >= 3) 
marquee_ability = (df['ability'] >= np.percentile(df['ability'], 97)) 
df.loc[marquee_age_gk & marquee_contract & marquee_ability & (df['position'] == 'GK'), 'low_proba'] = 1

# separate out current team and transfer pool and get more team stats
df_team = df[df['team'] == current_team].copy()
current_team_size = len(df_team)
max_wage_budget = df_team['wage'].sum() * 1.05
max_mean_team_age = df_team['age'].mean()
df_pool = df[df['team'] != current_team].copy()

# current team info
df_team

,name,age,country,position,overall,potential,team,start,expiration,value,wage,ability,old,low_proba
7,Cristiano Ronaldo (7),36,Portugal,CF,91,91,Manchester United,2021,2023,45000000.0,270000.0,91.0,1,0
32,Bruno Fernandes (32),26,Portugal,CAM,87,88,Manchester United,2020,2025,94000000.0,210000.0,87.0,0,1
37,De Gea (37),30,Spain,GK,87,87,Manchester United,2011,2023,53000000.0,150000.0,87.0,0,0
54,R. Varane (54),28,France,CB,86,88,Manchester United,2021,2025,68500000.0,180000.0,86.0,0,1
57,P. Pogba (57),28,France,CM,86,86,Manchester United,2016,2022,68000000.0,200000.0,86.0,0,0
67,J. Sancho (67),21,England,LM,85,89,Manchester United,2021,2026,78500000.0,130000.0,87.0,0,1
102,M. Rashford (102),23,England,LM,84,88,Manchester United,2014,2023,60500000.0,140000.0,86.0,0,0
129,E. Cavani (129),34,Uruguay,CF,84,84,Manchester United,2020,2022,20500000.0,175000.0,84.0,1,0
212,A. Wan-Bissaka (212),23,England,RB,82,85,Manchester United,2019,2024,38500000.0,105000.0,83.5,0,0
240,L. Shaw (240),25,England,LB,82,84,Manchester United,2014,2023,35000000.0,115000.0,82.0,0,0


In [4]:
# print summary of parameters
print('Summary of parameters:')
print(f'Team = {current_team} ({team_country})')
print(f'Current year = {current_year}')
print(f'Max team size = {max_team_size}')
print(f'Max number of foreign players = {max_foreign_size}')
print(f'Max transfer in = {max_in}')
print(f'Transfer budget = {max_transfer_budget}')
print(f'Wage budget = {max_wage_budget}')
print('Max mean age of team = {:.2f}'.format(max_mean_team_age))
print(f'Max transfer in, age >= 32 = {max_old}')
print('Current team: {} players, sum utility = {:.2f}, mean utility = {:.2f}'
      .format(len(df_team), df_team['ability'].sum(), df_team['ability'].mean()))
print('Current team: {} players have a low probability of transfer'.format(df_team['low_proba'].sum()))

print('\nPlayer transfer pool (excluding current team): {} players'.format(len(df_pool)))

Summary of parameters:
Team = Manchester United (England)
Current year = 2022
Max team size = 25
Max number of foreign players = 17
Max transfer in = 6
Transfer budget = 30000000.0
Wage budget = 3060750.0
Max mean age of team = 27.38
Max transfer in, age >= 32 = 1
Current team: 24 players, sum utility = 1966.50, mean utility = 81.94
Current team: 5 players have a low probability of transfer

Player transfer pool (excluding current team): 10011 players


In [5]:
# data prep for model
# transfer value, position, ability, wage 
dict_value_pool = dict(zip(df_pool['name'], df_pool['value']))
dict_position_pool = dict(zip(df_pool['name'], df_pool['position']))
dict_ability_pool = dict(zip(df_pool['name'], df_pool['ability']))
dict_wage_pool = dict(zip(df_pool['name'], df_pool['wage']))

dict_value_team = dict(zip(df_team['name'], df_team['value']))
dict_position_team = dict(zip(df_team['name'], df_team['position']))
dict_ability_team = dict(zip(df_team['name'], df_team['ability']))
dict_wage_team = dict(zip(df_team['name'], df_team['wage']))

# nationality
df_team['country'] = np.where(df_team['country'] == team_country, 0, 1)
df_pool['country'] = np.where(df_pool['country'] == team_country, 0, 1)
dict_country_pool = dict(zip(df_pool['name'], df_pool['country']))
dict_country_team = dict(zip(df_team['name'], df_team['country']))

# age
dict_age_pool = dict(zip(df_pool['name'], df_pool['age']))
dict_age_team = dict(zip(df_team['name'], df_team['age']))

# low_proba 
dict_lowproba_pool = dict(zip(df_pool['name'], df_pool['low_proba']))
dict_lowproba_team = dict(zip(df_team['name'], df_team['low_proba']))

# old players
dict_old_pool = dict(zip(df_pool['name'], df_pool['old']))

In [6]:
# define variables
x = LpVariable.dicts(name='b', indices=df_pool['name'],
                     lowBound=0, upBound=1, cat='Integer')
y = LpVariable.dicts(name='s', indices=df_team['name'],
                     lowBound=0, upBound=1, cat='Integer')

# define the problem
prob = LpProblem('Problem', LpMaximize)

# precompute linear combinations
# transfer value, ability, wage 
sum_value_pool = lpSum([dict_value_pool[kk] * x[kk] for kk in dict_value_pool])
sum_wage_pool = lpSum([dict_wage_pool[kk] * x[kk] for kk in dict_wage_pool])
sum_ability_pool = lpSum([dict_ability_pool[kk] * x[kk] for kk in dict_ability_pool])

sum_value_team = lpSum([dict_value_team[kk] * y[kk] for kk in dict_value_team])
sum_wage_team = lpSum([dict_wage_team[kk] * (1-y[kk]) for kk in dict_wage_team])
sum_ability_team = lpSum([dict_ability_team[kk] * (1-y[kk]) for kk in dict_ability_team])

# position
df_pos_pool = pd.get_dummies(df_pool['position'])
position_names_pool = df_pos_pool.columns
position_vector_pool = df_pos_pool.apply(lambda x: x.to_list(), axis=1).to_list()
dict_position_pool = makeDict([position_names_pool, df_pool['name']], np.array(position_vector_pool).T)

df_pos_team = pd.get_dummies(df_team['position'])
df_pos_team = df_pos_team.T.reindex(position_names_pool).T.fillna(0)
position_names_team = df_pos_team.columns
position_vector_team = df_pos_team.apply(lambda x: x.to_list(), axis=1).to_list()
dict_position_team = makeDict([position_names_team, df_team['name']], np.array(position_vector_team).T)

dict_position_need = {kk:0 for kk in position_names_pool}
dict_position_need['CF'] = 2
dict_position_need['LM'] = 1
dict_position_need['CM'] = 1
dict_position_need['CAM'] = 1
dict_position_need['CDM'] = 1
dict_position_need['RM'] = 1
dict_position_need['LB'] = 1
dict_position_need['RB'] = 1
dict_position_need['CB'] = 2
dict_position_need['GK'] = 2
assert sum(list(dict_position_need.values())) <= max_team_size, 'position needs infeasible!'

dict_position_max = {kk:0 for kk in position_names_pool}
dict_position_max['CF'] = 5
dict_position_max['LM'] = 5
dict_position_max['CM'] = 5
dict_position_max['CAM'] = 5
dict_position_max['CDM'] = 5
dict_position_max['RM'] = 5
dict_position_max['LB'] = 5
dict_position_max['RB'] = 5
dict_position_max['CB'] = 5

# nationality
sum_country_team = lpSum([dict_country_team[kk] * (1-y[kk]) for kk in dict_country_team])
sum_country_pool = lpSum([dict_country_pool[kk] * x[kk] for kk in dict_country_pool])

# age
sum_age_pool = lpSum([dict_age_pool[kk] * x[kk] for kk in dict_age_pool])
sum_age_team = lpSum([dict_age_team[kk] * (1-y[kk]) for kk in dict_age_team])

# low proba
sum_lowproba_pool = lpSum([dict_lowproba_pool[kk] * x[kk] for kk in dict_lowproba_pool])
sum_lowproba_team = lpSum([dict_lowproba_team[kk] * y[kk] for kk in dict_lowproba_team])

# old players
sum_old_pool = lpSum([dict_old_pool[kk] * x[kk] for kk in dict_old_pool])

# define constraints
# C1: maximum number of players in the team
prob += lpSum(x) - lpSum(y) <= max_team_size - current_team_size

# C2: maximum number of foreign players in the team
prob += sum_country_pool + sum_country_team <= max_foreign_size 

# C3: center forward, 2 <= n <= 5
# C4: left midfielder, 1 <= n <= 5
# C5: center midfielder, 1 <= n <= 5
# C6: center attacking midfielder, 1 <= n <= 5
# C7: center defensive midfielder, 1 <= n <= 5
# C8: right midfielder, 1 <= n <= 5
# C9: left back, 1 <= n <= 5
# C10: right back, 1 <= n <= 5
# C11: center back, 2 <= n <= 5
# C12: goalkeeper: n == 2

# minimum number of players in each position (all positions except goalkeeper)
for i in [xx for xx in dict_position_pool if xx != 'GK']: # all positions but GK
    prob += (
        lpSum(dict_position_pool[i][p] * x[p] for p in dict_position_pool[i]) + 
        lpSum(dict_position_team[i][p] * (1-y[p]) for p in dict_position_team[i])
        >= dict_position_need[i]
    )

# number of players at goalkeeper
for i in ['GK']: # restriction on GK
    prob += (
        lpSum(dict_position_pool[i][p] * x[p] for p in dict_position_pool[i]) + 
        lpSum(dict_position_team[i][p] * (1-y[p]) for p in dict_position_team[i])
        == dict_position_need[i]
    )

# maximum number of players in each position (all positions except goalkeeper)
for i in [xx for xx in dict_position_pool if xx != 'GK']: # all positions but GK
    prob += (
        lpSum(dict_position_pool[i][p] * x[p] for p in dict_position_pool[i]) + 
        lpSum(dict_position_team[i][p] * (1-y[p]) for p in dict_position_team[i])
        <= dict_position_max[i]
    )

# C13: do not consider low probability transfers (explained above)
prob += sum_lowproba_pool + sum_lowproba_team == 0    
    
# C14: maximum number of "old" players
prob += sum_old_pool <= max_old

# C15: mean age of the team should not increase after transfers
prob += (sum_age_pool + sum_age_team) <= max_mean_team_age * (current_team_size + lpSum(x) - lpSum(y))

# C16: transfer fee budget constraint
prob += sum_value_pool - sum_value_team <= max_transfer_budget 

# C17: wage constraint
prob += sum_wage_pool + sum_wage_team <= max_wage_budget

# C18: maximum number of transfers in   
prob += lpSum(x) <= max_in

# define objective function
prob += sum_ability_pool + sum_ability_team 

# solve the problem
prob.solve(PULP_CBC_CMD(msg=0))
print(f'status = {LpStatus[prob.status]}')

# print the results
for variable in prob.variables():
    if variable.varValue != 0:
        print(f'{variable.name} = {variable.varValue}')
    
print(f'Objective = {value(prob.objective)}')

status = Optimal
b_Azpilicueta_(187) = 1.0
b_G._Krychowiak_(466) = 1.0
b_K._Walker_(86) = 1.0
b_M._Neuer_(10) = 1.0
b_N._Madueke_(872) = 1.0
b_R._Gravenberch_(673) = 1.0
s_D._Henderson_(395) = 1.0
s_Juan_Mata_(1122) = 1.0
s_P._Jones_(2124) = 1.0
s_P._Pogba_(57) = 1.0
s_T._Heaton_(1753) = 1.0
Objective = 2076.5


In [7]:
# final team
final_team = []
for variable in prob.variables():
    if (variable.name.startswith('s')) & (variable.varValue == 0):
        final_team.append(variable.name)
    if (variable.name.startswith('b')) & (variable.varValue == 1):
        final_team.append(variable.name)
final_team = [xx[2:].replace('_', ' ') for xx in final_team]

name_map = dict(zip(df.loc[df['name'].str.contains('-'), 'name'].str.replace('-', ' ').tolist(),
                    df.loc[df['name'].str.contains('-'), 'name'].tolist()))
final_team = [xx if xx not in name_map else name_map[xx] for xx in final_team]

df_final = df[df['name'].isin(final_team)]
print(df_final.shape)
df_final

(25, 14)


,name,age,country,position,overall,potential,team,start,expiration,value,wage,ability,old,low_proba
7,Cristiano Ronaldo (7),36,Portugal,CF,91,91,Manchester United,2021,2023,45000000.0,270000.0,91.0,1,0
10,M. Neuer (10),35,Germany,GK,90,90,FC Bayern München,2011,2023,13500000.0,86000.0,90.0,1,0
32,Bruno Fernandes (32),26,Portugal,CAM,87,88,Manchester United,2020,2025,94000000.0,210000.0,87.0,0,1
37,De Gea (37),30,Spain,GK,87,87,Manchester United,2011,2023,53000000.0,150000.0,87.0,0,0
54,R. Varane (54),28,France,CB,86,88,Manchester United,2021,2025,68500000.0,180000.0,86.0,0,1
67,J. Sancho (67),21,England,LM,85,89,Manchester United,2021,2026,78500000.0,130000.0,87.0,0,1
86,K. Walker (86),31,England,RB,85,85,Manchester City,2017,2024,39000000.0,170000.0,85.0,0,0
102,M. Rashford (102),23,England,LM,84,88,Manchester United,2014,2023,60500000.0,140000.0,86.0,0,0
129,E. Cavani (129),34,Uruguay,CF,84,84,Manchester United,2020,2022,20500000.0,175000.0,84.0,1,0
187,Azpilicueta (187),31,Spain,CB,83,83,Chelsea,2012,2022,25000000.0,125000.0,83.0,0,0


In [8]:
# player summary
set1 = set(df_team['name'])
set2 = set(df_final['name'])
df_player_summary = pd.concat([pd.DataFrame({'out':sorted(set1-set2)}),
                               pd.DataFrame({'in':sorted(set2-set1)}),
                              ], axis=1) 
df_player_summary

,out,in
0,D. Henderson (395),Azpilicueta (187)
1,Juan Mata (1122),G. Krychowiak (466)
2,P. Jones (2124),K. Walker (86)
3,P. Pogba (57),M. Neuer (10)
4,T. Heaton (1753),N. Madueke (872)
5,NaN,R. Gravenberch (673)


In [9]:
# team comparison
tmp = []
for xx in [df_team, df_final]:
    if xx['country'].dtype=='int':
        dict1 = {'team_size':len(xx), 'foreign':np.sum(xx['country'] == 1), 'age':xx['age'].mean(),
            'ability_sum':xx['ability'].sum(), 'ability_mean':xx['ability'].mean()}
    else:
        dict1 = {'team_size':len(xx), 'foreign':np.sum(xx['country'] != team_country), 'age':xx['age'].mean(),
                 'ability_sum':xx['ability'].sum(), 'ability_mean':xx['ability'].mean()}
    dict2 = xx['position'].value_counts().to_dict()
    tmp.append({**dict1, **dict2})
df_team_summary = pd.DataFrame(tmp)
df_team_summary['status'] = ['current', 'transfer']
df_team_summary

,team_size,foreign,age,ability_sum,ability_mean,CB,CAM,GK,LM,CDM,CF,RB,LB,CM,RM,status
0,24,15,27.375,1966.5,81.9375,5,3,3,3,3,2,2,2,1,NaN,current
1,25,17,26.960,2076.5,83.0600,5,2,2,3,4,2,3,2,1,1.0,transfer


In [10]:
# ability, fee, wage breakdown
df_transfer_summary = pd.DataFrame({'ability_add':sum_ability_pool.value(),
                                    'ability_keep':sum_ability_team.value(),
                                    'ability_total':sum_ability_pool.value() + sum_ability_team.value(),
                                    'transfer_expenditure':sum_value_pool.value(),
                                    'transfer_income':sum_value_team.value(),
                                    'transfer_net':sum_value_pool.value() - sum_value_team.value(),
                                    'wage_add':sum_wage_pool.value(),
                                    'wage_keep':sum_wage_team.value(),
                                    'wage_total':sum_wage_pool.value() + sum_wage_team.value()},
                                  index=[0])
df_transfer_summary

,ability_add,ability_keep,ability_total,transfer_expenditure,transfer_income,transfer_net,wage_add,wage_keep,wage_total
0,504.5,1572.0,2076.5,133500000.0,104125000.0,29375000.0,403000.0,2437000.0,2840000.0
